# Check outputs

## Setup
#### Load the API key and relevant Python libaries.

In [7]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

In [9]:
def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0, max_tokens=500):
    response = openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
        max_tokens=max_tokens, # the maximum number of tokens the model can ouptut 
    )
    return response.choices[0].message.content

### Check output for potentially harmful content
우리는 이미 입력을 평가하는 맥락에서 조정(moderation) API에 대해 논의했습니다. 이제 **출력을 확인하는 맥락**에서 다시 살펴보겠습니다. 조정 API는 시스템 자체에서 생성된 출력을 필터링하고 조정하는 데에도 사용할 수 있습니다. 예를 들어, 여기에 사용자에게 생성된 응답이 있습니다 그리고 이전 비디오에서 본 것과 같은 방식으로 조정 API를 사용할 것입니다. 이 출력에 플래그가 지정되었는지 살펴보겠습니다. 그럼, 보시다시피, 이 출력은 플래그가 지정되지 않았고 모든 범주에서 매우 낮은 점수를 받았습니다. 응답을 고려하면 당연한 일입니다. 일반적으로, 출력을 확인하는 것이 중요할 수 있습니다. 예를 들어, 민감한 내용에 대한 챗봇을 만드는 경우, 출력 플래그 지정에 더 낮은 임계값을 사용할 수 있습니다. 조정 출력에서 ​​콘텐츠가 플래그 지정되었음을 나타내는 경우 적절한 조치를 취할 수 있습니다..
er.

In [12]:
final_response_to_customer = f"""
The SmartX ProPhone has a 6.1-inch display, 128GB storage, \
12MP dual camera, and 5G. The FotoSnap DSLR Camera \
has a 24.2MP sensor, 1080p video, 3-inch LCD, and \
interchangeable lenses. We have a variety of TVs, including \
the CineView 4K TV with a 55-inch display, 4K resolution, \
HDR, and smart TV features. We also have the SoundMax \
Home Theater system with 5.1 channel, 1000W output, wireless \
subwoofer, and Bluetooth. Do you have any specific questions \
about these products or any other products we offer?
"""
response = openai.moderations.create(
    input=final_response_to_customer
)
moderation_output = response.results[0]

print(moderation_output)

Moderation(categories=Categories(harassment=False, harassment_threatening=False, hate=False, hate_threatening=False, illicit=None, illicit_violent=None, self_harm=False, self_harm_instructions=False, self_harm_intent=False, sexual=False, sexual_minors=False, violence=False, violence_graphic=False, self-harm=False, sexual/minors=False, hate/threatening=False, violence/graphic=False, self-harm/intent=False, self-harm/instructions=False, harassment/threatening=False), category_applied_input_types=None, category_scores=CategoryScores(harassment=2.696166302484926e-05, harassment_threatening=9.87596831691917e-06, hate=7.229043148981873e-06, hate_threatening=2.0055701952514937e-06, illicit=None, illicit_violent=None, self_harm=1.2812188288080506e-06, self_harm_instructions=3.672591049053153e-07, self_harm_intent=2.012526920225355e-06, sexual=0.00015211118443403393, sexual_minors=1.154503297584597e-05, violence=0.0002972284273710102, violence_graphic=1.5082588106452022e-05, self-harm=1.2812188

### Check if output is factually based on the provided product information

출력을 확인하는 또 다른 방법은 **생성된 출력이 만족스러운지**, 그리고 **당신이 정의한 특정 기준에 맞는지** 모델 자체에 묻는 것입니다. 
생성된 출력을 모델에 대한 입력의 일부로 제공하고 출력의 품질을 평가하도록 요청하여 이를 수행할 수 있습니다.
다양한 방법으로 이를 수행할 수 있습니다. 그럼 예를 들어 보겠습니다.

In [15]:
system_message = f"""
You are an assistant that evaluates whether \
customer service agent responses sufficiently \
answer customer questions, and also validates that \
all the facts the assistant cites from the product \
information are correct.
The product information and user and customer \
service agent messages will be delimited by \
3 backticks, i.e. ```.
Respond with a Y or N character, with no punctuation:
Y - if the output sufficiently answers the question \
AND the response correctly uses product information
N - otherwise

Output a single letter only.
"""
customer_message = f"""
tell me about the smartx pro phone and \
the fotosnap camera, the dslr one. \
Also tell me about your tvs"""
product_information = """{ "name": "SmartX ProPhone", "category": "Smartphones and Accessories", "brand": "SmartX", "model_number": "SX-PP10", "warranty": "1 year", "rating": 4.6, "features": [ "6.1-inch display", "128GB storage", "12MP dual camera", "5G" ], "description": "A powerful smartphone with advanced camera features.", "price": 899.99 } { "name": "FotoSnap DSLR Camera", "category": "Cameras and Camcorders", "brand": "FotoSnap", "model_number": "FS-DSLR200", "warranty": "1 year", "rating": 4.7, "features": [ "24.2MP sensor", "1080p video", "3-inch LCD", "Interchangeable lenses" ], "description": "Capture stunning photos and videos with this versatile DSLR camera.", "price": 599.99 } { "name": "CineView 4K TV", "category": "Televisions and Home Theater Systems", "brand": "CineView", "model_number": "CV-4K55", "warranty": "2 years", "rating": 4.8, "features": [ "55-inch display", "4K resolution", "HDR", "Smart TV" ], "description": "A stunning 4K TV with vibrant colors and smart features.", "price": 599.99 } { "name": "SoundMax Home Theater", "category": "Televisions and Home Theater Systems", "brand": "SoundMax", "model_number": "SM-HT100", "warranty": "1 year", "rating": 4.4, "features": [ "5.1 channel", "1000W output", "Wireless subwoofer", "Bluetooth" ], "description": "A powerful home theater system for an immersive audio experience.", "price": 399.99 } { "name": "CineView 8K TV", "category": "Televisions and Home Theater Systems", "brand": "CineView", "model_number": "CV-8K65", "warranty": "2 years", "rating": 4.9, "features": [ "65-inch display", "8K resolution", "HDR", "Smart TV" ], "description": "Experience the future of television with this stunning 8K TV.", "price": 2999.99 } { "name": "SoundMax Soundbar", "category": "Televisions and Home Theater Systems", "brand": "SoundMax", "model_number": "SM-SB50", "warranty": "1 year", "rating": 4.3, "features": [ "2.1 channel", "300W output", "Wireless subwoofer", "Bluetooth" ], "description": "Upgrade your TV's audio with this sleek and powerful soundbar.", "price": 199.99 } { "name": "CineView OLED TV", "category": "Televisions and Home Theater Systems", "brand": "CineView", "model_number": "CV-OLED55", "warranty": "2 years", "rating": 4.7, "features": [ "55-inch display", "4K resolution", "HDR", "Smart TV" ], "description": "Experience true blacks and vibrant colors with this OLED TV.", "price": 1499.99 }"""
q_a_pair = f"""
Customer message: ```{customer_message}```
Product information: ```{product_information}```
Agent response: ```{final_response_to_customer}```

Does the response use the retrieved information correctly?
Does the response sufficiently answer the question

Output Y or N
"""
messages = [
    {'role': 'system', 'content': system_message},
    {'role': 'user', 'content': q_a_pair}
]

response = get_completion_from_messages(messages, max_tokens=1)
print(response)

Y


In [16]:
another_response = "life is like a box of chocolates"
q_a_pair = f"""
Customer message: ```{customer_message}```
Product information: ```{product_information}```
Agent response: ```{another_response}```

Does the response use the retrieved information correctly?
Does the response sufficiently answer the question?

Output Y or N
"""
messages = [
    {'role': 'system', 'content': system_message},
    {'role': 'user', 'content': q_a_pair}
]

response = get_completion_from_messages(messages)
print(response)

N


# Build an End-to-End System

여기에서는 코스 전체에서 본 프롬프트 체인을 구성합니다.

## System of chained prompts for processing the user query

이제는 앞에서 배운 모든 것을 모아 고객 서비스 지원의 처음부터 끝까지 예시를 만들어 보겠습니다. 
다음 단계를 살펴보겠습니다.
1. 입력을 확인하여 조정 API에 플래그가 지정되었는지 확인합니다.
2. 지정되지 않은 경우 제품 목록을 추출합니다.
3. 제품을 찾으면 검색을 시도합니다.
4. 모델을 사용하여 사용자 질문에 답을 합니다.
5. 마지막으로 조정 API에 답변을 입력합니다. 플래그가 지정되 않으면 다음으로 진행합니다.
6. 응답이 초기 사용자 질의에 잘 답하는 지를 모델에 질문합니다.
7. Yes 인 경우엔 사용자에게 반환합니다.  No 인 경우엔 사용자를 사람과 연결한다고 말합니다.

In [25]:
import utils

import panel as pn  # GUI
pn.extension()

In [26]:
def process_user_message(user_input, all_messages, debug=True):
    delimiter = '```'
    
    # Step 1: Check input to see if it flags the Moderation API or is a prompt injection
    response = openai.moderations.create(input=user_input)
    moderation_output = response.results[0]

    if moderation_output.flagged:
        print("Step 1: Input flagged by Moderation API.")
        return "Sorry, we cannot process this request."

    if debug: print("Step 1: Input passed moderation check.")
    
    category_and_product_response = utils.find_category_and_product_only(user_input, utils.get_products_and_category())
    # print(category_and_product_response)
    
    # Step 2: Extract the list of products
    category_and_product_list = utils.read_string_to_list(category_and_product_response)
    # print(category_and_product_list)

    if debug: print("Step 2: Extracted list of products.")

    # Step 3: If products are found, look them up
    product_information = utils.generate_output_string(category_and_product_list)
    # print(product_information)
    
    if debug: print("Step 3: Looked up product information.")

    # Step 4: Answer the user question
    system_message = f"""
    You are a customer service assistant for a large electronic store. \
    Respond in a friendly and helpful tone, with concise answers. \
    Make sure to ask the user relevant follow-up questions.
    """
    messages = [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': f"{delimiter}{user_input}{delimiter}"},
        {'role': 'assistant', 'content': f"Relevant product information:\n{product_information}"}
    ]

    final_response = get_completion_from_messages(all_messages + messages)
    if debug:print("Step 4: Generated response to user question.")
    all_messages = all_messages + messages[1:]

    # Step 5: Put the answer through the Moderation API
    response = openai.moderations.create(input=final_response)
    moderation_output = response.results[0]

    if moderation_output.flagged:
        if debug: print("Step 5: Response flagged by Moderation API.")
        return "Sorry, we cannot provide this information."

    if debug: print("Step 5: Response passed moderation check.")

    # Step 6: Ask the model if the response answers the initial user query well
    user_message = f"""
    Customer message: {delimiter}{user_input}{delimiter}
    Agent response: {delimiter}{final_response}{delimiter}

    Does the response sufficiently answer the question?
    """
    messages = [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': user_message}
    ]
    evaluation_response = get_completion_from_messages(messages)
    if debug: print("Step 6: Model evaluated the response.")

    # Step 7: If yes, use this answer; if not, say that you will connect the user to a human
    if "Y" in evaluation_response:  # Using "in" instead of "==" to be safer for model output variation (e.g., "Y." or "Yes")
        if debug: print("Step 7: Model approved the response.")
        return final_response, all_messages
    else:
        if debug: print("Step 7: Model disapproved the response.")
        neg_str = "I'm unable to provide the information you're looking for. I'll connect you with a human representative for further assistance."
        return neg_str, all_messages

In [27]:
user_input = "tell me about the smartx pro phone and the fotosnap camera, the dslr one. Also what tell me about your tvs"
response,_ = process_user_message(user_input,[])
print(response)

Step 1: Input passed moderation check.
Step 2: Extracted list of products.
Step 3: Looked up product information.
Step 4: Generated response to user question.
Step 5: Response passed moderation check.
Step 6: Model evaluated the response.
Step 7: Model approved the response.
The SmartX ProPhone is a powerful smartphone with a 6.1-inch display, 128GB storage, 12MP dual camera, and 5G capability. It is priced at $899.99 with a 1-year warranty.

The FotoSnap DSLR Camera features a 24.2MP sensor, 1080p video recording, 3-inch LCD screen, and interchangeable lenses. Priced at $599.99, it comes with a 1-year warranty.

As for TVs, we have a variety of options:
1. CineView 4K TV (55-inch) with HDR and Smart TV features for $599.99 (2-year warranty).
2. CineView 8K TV (65-inch) with HDR and Smart TV features for $2999.99 (2-year warranty).
3. CineView OLED TV (55-inch) with 4K resolution, HDR, and Smart TV capabilities for $1499.99 (2-year warranty).

Do you have any specific questions about t

### Function that collects user and assistant messages over time

In [32]:
def collect_messages(debug=False):
    user_input = inp.value_input
    if debug: print(f"User Input = {user_input}")
    if user_input == "":
        return
    inp.value = ''
    global context
    #response, context = process_user_message(user_input, context, utils.get_products_and_category(),debug=True)
    response, context = process_user_message(user_input, context, debug=True)
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(user_input, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600)))
        # pn.Row('Assistant:', pn.pane.Markdown(response, width=600, style={'background-color': '#F6F6F6'})))
 
    return pn.Column(*panels)

### Chat with the chatbot!
시스템 메시지에는 OrderBot이 수행해야 하는 작업에 대한 자세한 지침이 포함되어 있습니다.

In [35]:
panels = [] # collect display 

context = [ {'role':'system', 'content':"You are Service Assistant"} ]  

inp = pn.widgets.TextInput(placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Service Assistant")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard

# Sample Chats ...
# What TVs do you have?
# Which is the cheapest?
# Which is the most expensive?
# Tell me more about it.

Column
    [0] TextInput(placeholder='Enter text here…')
    [1] Row
        [0] Button(name='Service Assistant')
    [2] ParamFunction(function, _pane=Str, defer_load=False, height=300, loading_indicator=True)